In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.framework import ops

from load_data import *
from models.customlayers import *
from models.activations import *
from training import *

In [ ]:
data_dir = os.path.expanduser('~/Insight/video-representations/frames')
training_file = os.path.join(data_dir, 'training.tfrecords')
validation_file = os.path.join(data_dir, 'validation.tfrecords')
testing_file = os.path.join(data_dir, 'testing.tfrecords')

In [3]:
N_EPOCHS = 4
HEIGHT = 60
WIDTH = 80
N_CHANNELS = 3
SEQLEN = 100
QUEUE_THREADS = 4

def read_single_record(filepath_queue):
    _, serial = reader.read(filepath_queue)
    
    features = {
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'video': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
        'length': tf.FixedLenFeature([], tf.int64)
    }
    
    features = tf.parse_single_example(serial, features = features)
    video = tf.decode_raw(features['video'], tf.uint8)
    video = tf.cast(tf.reshape(video, (-1, HEIGHT, WIDTH, N_CHANNELS)), tf.float32)
    
    return video
    
    
with tf.name_scope('training_queue'):
    filepath_queue = tf.train.string_input_producer([training_file])
    training_video = read_single_record(filepath_queue)
    
with tf.name_scope('shuffle_queue'):
    shuffle_queue = tf.RandomShuffleQueue(capacity=64, min_after_dequeue=16, dtype=tf.float32)
    shuffle_enqueue_op = shuffle_queue.enqueue(training_video)
    
with tf.name_scope('padding_queue'):
    padding_queue = tf.PaddingFIFOQueue(capacity=256, dtype=tf.float32, shapes=(None, HEIGHT, WIDTH, N_CHANNELS))
    padding_enqueue_op = padding_queue.enqueue(shuffle_queue.dequeue())
    
runner = tf.train.QueueRunner(padding_queue, [padding_enqueue_op] * QUEUE_THREADS)    
coord = tf.train.Coordinator()

with tf.Session() as sesh:
    enqueue_threads = runner.create_threads(sesh, coord=coord, start=True)
    
    for i in range(N_EPOCHS):
        if coord.should_stop():
            break
        
        sesh.run(padding_queue.dequeue())
        
    coord.request_stop()
    coord.join(enqueue_threads)

NameError: name 'training_file' is not defined